In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit 
from sklearn.preprocessing import OneHotEncoder

from Bio import pairwise2
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.metrics import r2_score

from sklearn import preprocessing
from Bio.SeqUtils.ProtParam import ProteinAnalysis


from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
import pandascharm as pc

from Bio.SubsMat.MatrixInfo import blosum62
from Bio.SubsMat.MatrixInfo import blosum45

In [ ]:
def removeoutlier_col(df,cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df[[cols]] < (Q1 - 1.5 * IQR)) |(df[[cols]] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out


def RF(X_train, y_train, X_val):
    
    rf = RandomForestRegressor()
    rf.fit(X_train,np.ravel(y_train))
    RF_pred = rf.predict(X_val)

    return RF_pred

def XGBR(X_train, y_train, X_val):
    
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    XGB_pred = model.predict(X_val)
    
    return XGB_pred

def SVM(X_train, y_train, X_val):
    
    model = SVR()
    model.fit(X_train, y_train)
    SVM_pred = model.predict(X_val)
    
    return SVM_pred



In [ ]:
blosum62.update(((b,a),val) for (a,b),val in list(blosum62.items()))
blosum45.update(((b,a),val) for (a,b),val in list(blosum45.items()))

def score_pairwise(seq1, seq2, matrix, gap_s, gap_e, gap = True):
    for A,B in zip(seq1, seq2):
        diag = ('-'==A) or ('-'==B)
        yield (gap_e if gap else gap_s) if diag else matrix[(A,B)]
        gap = diag

In [ ]:
def encode(encoding, output, df_clean, aln, esm1b, key = None):
    

    ClustalAlign = AlignIO.read(aln, 'clustal')
    summary_align = AlignInfo.SummaryInfo(ClustalAlign )
    dframe = pc.from_bioalignment(ClustalAlign).transpose()
    sequences = dframe.loc[df_clean.index]
    
    y = df_clean['Log'+output]

    scaler = preprocessing.StandardScaler()
    
    
    if encoding == 'One-Hot-Encoder':

        one_hot = OneHotEncoder()
        encoded = one_hot.fit(sequences)
        X = encoded.transform(sequences).toarray()
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
           
    if encoding == 'Bag-of-Words':

        X = pd.DataFrame([ProteinAnalysis(i).count_amino_acids() for i in df_clean['Sequence']])
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
    if encoding == 'bigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                st = i+j
                all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-1):
                try:
                    check = temp[k] + temp[k+1]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
    
    if encoding == 'trigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    st = i+j+k
                    all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-2):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
        
    if encoding == 'quadrogram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    for l in lst:
                        st = i+j+k+l
                        all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-3):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]+temp[k+3]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)

    if encoding == 'BLOSUM62':

        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum62, -5, -1))
                j += 1
            i += 1
        
        X = np.array(enc_seq)
        scaler.fit(enc_seq)
        X_scaled = scaler.transform(enc_seq)
        
        
        
    if encoding == 'BLOSUM45':
        
        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum45, -5, -1))
                j += 1
            i += 1
        
        X = np.array(enc_seq)   
        scaler.fit(enc_seq)
        X_scaled = scaler.transform(enc_seq)
        
    if encoding == 'ESM1b':
        

        encoded = esm1b.loc[df_clean.index]
        X = np.array(encoded)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
    return X, y, X_scaled, scaler


In [ ]:
def evaluate_performance_identity_based(enzyme , df, output, aln, esm1b, methods, state):

    summary=pd.DataFrame()
    
    df['Log'+output]=np.log10(df[output])
    
    df_clean = removeoutlier_col(df,'Log'+output).copy()
    
    # Create a mapping of unique sequences to unique codes
    sequence_to_code = {seq: f"ENZYME_{i+1}" for i, seq in enumerate(df_clean['Sequence'].unique())}

    # Map these codes to a new column in the DataFrame using .loc
    df_clean.loc[:, 'Sequence Code'] = df_clean['Sequence'].map(sequence_to_code)
    
    df_clean = df_clean.set_index('Index')

        
    for method in methods:

        X, y, X_scaled, scaler = encode(method, output, df_clean, aln, esm1b, key=None)
        
        splitter = GroupShuffleSplit(test_size=.20, n_splits=10, random_state = state)

        split = splitter.split(df_clean, groups=df_clean['Sequence Code'])
        
        train_inds, val_inds = next(split)
        
        X_train = X[train_inds]
        y_train = y[train_inds]
        X_scaled_train = X_scaled[train_inds]
        
        
        X_val = X[val_inds]
        y_val = y[val_inds]
        X_scaled_val = X_scaled[val_inds]

        
        y_predicted_RF = RF(X_train, y_train, X_val)
        y_predicted_XGBR = XGBR(X_train, y_train, X_val)
        y_predicted_SVM = SVM(X_scaled_train, y_train, X_scaled_val)

        
        summary['RF with ' + method] = y_predicted_RF
        summary['XGB with ' + method] = y_predicted_XGBR
        summary['SVM with ' + method] = y_predicted_SVM


    X_train_seq = df_clean.loc[y_train.index]['Sequence']
    X_val_seq = df_clean.loc[y_val.index]['Sequence']
    max_list = []
    
    for sequence_val in X_val_seq:
        score_list=[]
        for sequence_train in X_train_seq:
            alignment = pairwise2.align.globalxx(sequence_val, sequence_train)[0]
            aligned_length = len(alignment.seqA)
            identical_positions = sum(a == b for a,b in zip(alignment.seqA, alignment.seqB))
            identity_score = (identical_positions / aligned_length)
            score_list.append(identity_score)
        max_list.append(max(score_list))

 
    # Define the ranges
    ranges = [(0, 0.7999), (0.80, 0.9999), (1, 1)]

    # Initialize a dictionary to hold the counts for each range
    counts = {r: 0 for r in ranges}

    # Iterate through each element and determine which range it falls into
    for element in max_list:
        for r in ranges:
            if r[0] <= element <= r[1]:
                counts[r] += 1

    # Print the counts for each range
    for r in ranges:
        print(f"Range {r}: {counts[r]}")

    summary['y_val'] = y_val.values
    summary['Sequence Identity'] = max_list
    summary['wild type or mutant'] = df_clean.loc[y_val.index]['wild type or mutant'].values
    summary=summary.set_index(y_val.index)

    identity_less100 = summary[summary['Sequence Identity'] < 1] 
    identity_80 = summary[(summary['Sequence Identity'] < 0.9999) & (summary['Sequence Identity'] > 0.8)]
    identity_0 = summary[summary['Sequence Identity'] < 0.7999]
    identity_80_99 = summary[(summary['Sequence Identity'] <= 0.99) & (summary['Sequence Identity'] > 0.8)]
    identity_99 = summary[summary['Sequence Identity'] > 0.99] 

    identity_less100_w = summary[(summary['Sequence Identity'] < 1) & (summary['wild type or mutant'] == 'wild')] 
    identity_80_w = summary[(summary['Sequence Identity'] < 0.9999) & (summary['Sequence Identity'] > 0.8) &
                                                               (summary['wild type or mutant'] == 'wild')]
    identity_0_w = summary[(summary['Sequence Identity'] < 0.7999) & (summary['wild type or mutant'] == 'wild')]
    identity_80_99_w = summary[(summary['Sequence Identity'] <= 0.99) & (summary['Sequence Identity'] > 0.8) &
                                                                      (summary['wild type or mutant'] == 'wild')]
    identity_99_w = summary[(summary['Sequence Identity'] > 0.99) & (summary['wild type or mutant'] == 'wild')]  


    identity_less100_m = summary[(summary['Sequence Identity'] < 1) & (summary['wild type or mutant'] == 'mutant')] 
    identity_80_m = summary[(summary['Sequence Identity'] < 0.9999) & (summary['Sequence Identity'] > 0.8) &
                                                                    (summary['wild type or mutant'] == 'mutant')]
    identity_99_m = summary[(summary['Sequence Identity'] > 0.99) & (summary['wild type or mutant'] == 'mutant')]  




    matrix = [identity_less100, identity_80 ,identity_0 ,identity_80_99, identity_99 ,
              identity_less100_w, identity_80_w, identity_0_w, identity_80_99_w , identity_99_w,
              identity_less100_m,identity_80_m,identity_99_m]
    list_R2=[]

    for identity in matrix:
        for model in summary.columns[:-1]:
            r2=r2_score(identity['y_val'], identity[model])
            list_R2.append(r2)



    x = np.reshape(list_R2, (len(matrix), len(summary.columns[:-1])))

    number_of_data = [len(a) for a in matrix]

    Results=pd.DataFrame(x, columns=summary.columns[:-1], index = ['<100%', '80-99.99%', '0-79.99%', '80-99%', '>99%',
                                                              '<100% wild type','80-99.99% wild type', '0-79.99% wild type', '80-99% wild type', '>99% wild type',
                                                              '<100% mutant type','80-99.99% mutant type', '>99% mutant type'])
    Results['Number of Data'] = number_of_data
    
    return summary, list_R2, Results, number_of_data



In [ ]:
enzyme = 'betaGlucosidasewithMutants'

df = pd.read_excel('betaGlucosidasewithMutantsOptimumTemperature.xlsx')

output = 'pNP-Glc kcat/Km (1/smM)'
aln = enzyme + '.aln'

x = datetime.datetime.now()
date = str(x.year)+str(x.month)+str(x.day)


methods = ['ESM1b', 'BLOSUM45', 'BLOSUM62', 'bigram',  'quadrogram', 'Bag-of-Words']


esm1b = pd.read_excel(enzyme+'ESM1b_embeddings.xlsx', index_col = 0)

In [ ]:
df_kcatKmTopt = df[df['Percentage Activity Depending on Optimum Temp']==1]

In [ ]:
list_summary = []
list_list_R2 = []
list_Results = []
random_state = [202 , 1, 42, 101, 2022,5 , 10, 22, 1995, 0]

list_number_of_data = []

for state in random_state:
    summary, list_R2, Results, number_of_data = evaluate_performance_identity_based(enzyme , 
                                                                                  df_kcatKmTopt , output, aln, esm1b,
                                                                                  methods, state)
    print( "Run is completed")
    print(Results)
    list_summary.append(summary)
    list_list_R2.append(list_R2)
    list_Results.append(Results)
    list_number_of_data.append(number_of_data)

In [ ]:
df_average = pd.concat([list_Results[0],list_Results[1],list_Results[2],list_Results[3], list_Results[4],
                       list_Results[5],list_Results[6],list_Results[7],list_Results[8], list_Results[9]])

average = df_average.groupby(level=0, sort=False).mean()
std = df_average.groupby(level=0, sort=False).std()
list_Results.append(average)
list_Results.append(std)



In [ ]:
df_number_of_data = pd.DataFrame(data = list_number_of_data, columns = ['<100%', '80-99.99%', '0-79.99%', '80-99%', '>99%',
                                                              '<100% wild type','80-99.99% wild type', '0-79.99% wild type', '80-99% wild type', '>99% wild type',
                                                              '<100% mutant type','80-99.99% mutant type', '>99% mutant type'])
list_Results.append(df_number_of_data)